# Setup

In [1]:
import sys

sys.path.append("..")

In [2]:
# Main Constants
from pathlib import Path

# Base path for testing - adjust as needed
BASE_PATH = Path("temp").resolve()  # Points to anthropic_agent root
print(BASE_PATH)

/Users/aurosoni/conductor/workspaces/anthropic_agent/nagoya/anthropic_agent/common_tools/temp


# Tool Tests

## Tool: Read File

In [3]:
from anthropic_agent.common_tools.read_file import ReadFileTool

# Create the read file tool
read_file_tool = ReadFileTool(base_path=BASE_PATH)
read_file = read_file_tool.get_tool()

In [4]:
print(read_file("README.md"))

[lines 1-100 of 102 in README.md]
# anthropic-agent

A small Python library for building **Anthropic-powered agents** with:
- **streaming** responses
- **tool calling** (client tools)
- **optional server tools** (passed through to Anthropic)
- **persistence** (filesystem or Postgres) and **file storage** (local or S3)

> **For sample agent configurations examples hanve a look at agent_test.ipynb**

The primary package is `anthropic_agent` (install name: `anthropic-agent`).

## AnthropicAgent features

- **Resumable sessions**: pass `agent_uuid` to reload `messages`, `container_id`, token counters, and file registry from the DB backend.
- **Streaming output**: stream formatted chunks to an `asyncio.Queue` (`formatter="xml"` or `formatter="raw"`).
- **Tool execution loop**: executes *client tools* locally when the model returns `stop_reason == "tool_use"`, then feeds tool results back to the model.
- **Server tools passthrough**: include Anthropic server tools configs (e.g. code executio

## Tool: List Dir

In [5]:
from anthropic_agent.common_tools.list_dir import ListDirTool

list_dir_tool = ListDirTool(base_path=BASE_PATH)
list_dir = list_dir_tool.get_tool()

In [6]:
print(list_dir("."))

temp/
   - test-agent-123/
      - api_refactor_20260115_100908.md
      - API_v2_Upgrade_20260115_100908.md
      - db_migration_20260115_100908.md
      - plan_20260115_074238.md
      - plan_20260115_074249.md
   - patch_demo.md
   - README.md
   - test_file.md


## Tool: Glob File Search

In [7]:
from anthropic_agent.common_tools.glob_file_search import GlobFileSearchTool

glob_search_tool = GlobFileSearchTool(base_path=BASE_PATH)
glob_file_search = glob_search_tool.get_tool()

In [8]:
print(glob_file_search("*.md"))

patch_demo.md
test-agent-123/API_v2_Upgrade_20260115_100908.md
test-agent-123/db_migration_20260115_100908.md
test-agent-123/api_refactor_20260115_100908.md
test-agent-123/plan_20260115_074249.md
test-agent-123/plan_20260115_074238.md
README.md
test_file.md


## Tool: Grep Search

In [9]:
from anthropic_agent.common_tools.grep_search import GrepSearchTool

grep_search_tool = GrepSearchTool(base_path=BASE_PATH)
grep_search = grep_search_tool.get_tool()

In [10]:
print(grep_search("agent/|Test"))

README.md:
  7- - **persistence** (filesystem or Postgres) and **file storage** (local or S3)
  8- 
  9: > **For sample agent configurations examples hanve a look at agent_<match>test</match>.ipynb**
  10- 
  11- The primary package is `anthropic_agent` (install name: `anthropic-agent`).
  23- - **Files API + storage backends** (optional): detect generated `file_id`s, download via Anthropic Files API, and store via local filesystem or S3.
  24- 
  25: For a step-by-step diagram of the run loop, see `anthropic_<match>agent/</match>agent-flow.md`.
  26- 
  27- ## Requirements
  57- 
  58- Notes:
  59: - For local tool calling, pass `tools=[...]` (see `anthropic_<match>agent/</match>tools/sample_tools.py`).
  60- - Runs are persisted under `./data/` by default (agent config, conversation history, run logs).
  61- 
  65- 
  66- The demo app lives in `demos/fastapi_server/` and exposes:
  67: - `POST /<match>agent/</match>run`: streams output as Server-Sent Events (SSE)
  68: - `POST /<matc

## Tool: Apply Patch

In [3]:
from anthropic_agent.common_tools.apply_patch import ApplyPatchTool

apply_patch_tool = ApplyPatchTool(base_path=BASE_PATH)
apply_patch = apply_patch_tool.get_tool()

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/tools/base.py:441: UserWarning: ApplyPatchTool: Unknown docstring placeholder '"status"'. Available placeholders: ['max_patch_size_mb', 'max_file_size_mb']
  func.__doc__ = self._render_docstring()


In [4]:
from __future__ import annotations

import json
from pathlib import Path
from pprint import pprint

# We'll demonstrate ALL patch operation types on the same logical file.
# Because BASE_PATH is your "temp" folder, all paths below are relative to that.
DEMO_PATH = "patch_demo.md"
DEMO_MOVED_PATH = "patch_demo_moved.md"


def run_patch(title: str, patch_text: str, *, dry_run: bool = False) -> None:
    # print(f"\n--- {title} (dry_run={dry_run}) ---")
    pprint(json.loads(apply_patch(patch_text, dry_run=dry_run)))


# Ensure clean slate so "Add File" works
for p in [Path(BASE_PATH) / DEMO_PATH, Path(BASE_PATH) / DEMO_MOVED_PATH]:
    if p.exists():
        p.unlink()

print("Demo files prepared.")

Demo files prepared.


In [5]:
# 1) Add File
patch_add = """*** Begin Patch
*** Add File: patch_demo.md
+# Patch Demo
+
+Alpha
+Beta
+
+## Section
+- one
+- two
+
*** End Patch"""

run_patch("Add File", patch_add, dry_run=False)

{'dry_run': False,
 'hunks_applied': 0,
 'lines_added': 9,
 'lines_removed': 0,
 'op': 'add',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [6]:
# 2) Update File (simple hunk)
patch_update_simple = """*** Begin Patch
*** Update File: patch_demo.md
@@
 Alpha
-Beta
+Beta v2
*** End Patch"""

run_patch("Update File (simple)", patch_update_simple, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 1,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [7]:
# 3) Update File with a scope line (narrows matching to a subsection)
# NOTE: The scope line `@@ ## Section` anchors the search; don't repeat it as context
patch_update_scoped = """*** Begin Patch
*** Update File: patch_demo.md
@@ ## Section
 - one
-- two
+- two (updated)
*** End Patch"""

run_patch("Update File (scoped)", patch_update_scoped, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 1,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [8]:
# 4) Add a nested section so we can demo nested scopes
# Use EOF marker to append at end of file
patch_add_subsection = """*** Begin Patch
*** Update File: patch_demo.md
@@
 - two (updated)
+
+### Subsection
+note: hello
*** End of File
*** End Patch"""

run_patch("Update File (add subsection)", patch_add_subsection, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 3,
 'lines_removed': 0,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [9]:
# 5) Update File with nested scopes (class -> method style, but works for headings too)
# Nested scopes: @@ ## Section then @@ ### Subsection narrows to that subsection
patch_update_nested_scopes = """*** Begin Patch
*** Update File: patch_demo.md
@@ ## Section
@@ ### Subsection
-note: hello
+note: hello (edited)
*** End Patch"""

run_patch("Update File (nested scopes)", patch_update_nested_scopes, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 1,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [10]:
# 6) Update File targeting end-of-file (EOF marker)
# EOF marker ensures matching happens at the end of file
patch_update_eof = """*** Begin Patch
*** Update File: patch_demo.md
@@
 note: hello (edited)
+EOF line added
*** End of File
*** End Patch"""

run_patch("Update File (EOF marker)", patch_update_eof, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 0,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [11]:
# 7) Move/Rename during an update
patch_move = """*** Begin Patch
*** Update File: patch_demo.md
*** Move to: patch_demo_moved.md
@@
 # Patch Demo
+# (moved)
*** End Patch"""

run_patch("Update File + Move to", patch_move, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 0,
 'moved_from': 'patch_demo.md',
 'op': 'update',
 'path': 'patch_demo_moved.md',
 'status': 'ok'}


In [12]:
# 8) Delete File
patch_delete = """*** Begin Patch
*** Delete File: patch_demo_moved.md
*** End Patch"""

run_patch("Delete File", patch_delete, dry_run=False)

{'dry_run': False,
 'hunks_applied': 0,
 'lines_added': 0,
 'lines_removed': 14,
 'op': 'delete',
 'path': 'patch_demo_moved.md',
 'status': 'ok'}


In [13]:
# Optional: verify end state
print(list_dir("."))

NameError: name 'list_dir' is not defined

In [23]:
# Optional: inspect file content (should be gone after delete)
print(read_file("patch_demo_moved.md"))

Path does not exist: patch_demo_moved.md. Use glob_file_search to find files matching a pattern.


## Tool: Code Execution

This section demonstrates the `CodeExecutionTool`:
- Stateful execution (variables persist across calls)
- Using `embedded_tools` from within executed code
- `authorized_imports` allow/deny behavior
- Output truncation via `max_output_chars`
- Agent UUID injection patterns (manual + via `AnthropicAgent` when available)


In [24]:
from __future__ import annotations

from pathlib import Path

from anthropic_agent.common_tools.code_execution_tool import CodeExecutionTool
from anthropic_agent.tools import tool

CODE_OUTPUT_BASE = BASE_PATH / "code_tool_outputs"
CODE_OUTPUT_BASE.mkdir(parents=True, exist_ok=True)

print("CodeExecutionTool output base:", CODE_OUTPUT_BASE)


CodeExecutionTool output base: /Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs


In [25]:
# Define embedded tools callable from inside code

@tool
def add(a: float, b: float) -> str:
    """Add two numbers.

    Args:
        a: First number
        b: Second number
    """
    return str(a + b)


@tool
def to_upper(s: str) -> str:
    """Upper-case a string.

    Args:
        s: Input string
    """
    return s.upper()


In [26]:
# Instantiate CodeExecutionTool with embedded_tools + authorized_imports
# - json is NOT in the default BASE_BUILTIN_MODULES list, so we explicitly allow it.

code_tool = CodeExecutionTool(
    output_base_path=CODE_OUTPUT_BASE,
    embedded_tools=[add, to_upper],
    authorized_imports=["json"],
    max_output_chars=2_000,
)

# In real agent usage, the UUID is injected automatically.
# In a notebook, we set it manually.
code_tool.set_agent_uuid("notebook-demo-agent")

code_execute = code_tool.get_tool()

print("Tool name:", code_execute.__tool_schema__["name"])
print("Has __tool_instance__:", hasattr(code_execute, "__tool_instance__"))
print("Injected UUID:", code_tool.agent_uuid)


Tool name: code_execution
Has __tool_instance__: True
Injected UUID: notebook-demo-agent


In [27]:
# Scenario 1: Stateful execution (variables persist)
print(code_execute("""
x = 10
print('x initially:', x)
"""))

print(code_execute("""
# x is still defined here
x = x + 5
print('x after increment:', x)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123736_1e78dc8f.txt

x initially: 10

output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123736_9406f79a.txt

x after increment: 15



In [28]:
# Scenario 2: Call embedded tools from executed code
print(code_execute("""
print('add(2, 3) =', add(2, 3))
print('to_upper(hello) =', to_upper('hello'))
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123753_e0a42c7d.txt

add(2, 3) = 5
to_upper(hello) = HELLO



In [ ]:
# Scenario 3: Authorized import allowed (json was explicitly whitelisted)
print(code_execute("""
import json
payload = {"ok": True, "n": 123}
print(json.dumps(payload, sort_keys=True))
"""))


In [ ]:
# Scenario 3: Authorized import allowed (json was explicitly whitelisted)
print(code_execute("""
import json
payload = {"ok": True, "n": 123}
print(json.dumps(payload, sort_keys=True))
"""))


In [ ]:
# Scenario 3: Authorized import allowed (json was explicitly whitelisted)
print(code_execute("""
import json
payload = {"ok": True, "n": 123}
print(json.dumps(payload, sort_keys=True))
"""))


In [ ]:
# Scenario 3: Authorized import allowed (json was explicitly whitelisted)
print(code_execute("""
import json
payload = {"ok": True, "n": 123}
print(json.dumps(payload, sort_keys=True))
"""))


In [29]:
# Scenario 3: Authorized import allowed (json was explicitly whitelisted)
print(code_execute("""
import json
payload = {"ok": True, "n": 123}
print(json.dumps(payload, sort_keys=True))
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123756_fa7a2d81.txt

{"n": 123, "ok": true}



In [30]:
# Scenario 4: Unauthorized import blocked (os is not allowed)
print(code_execute("""
import os
print('this should not run')
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123756_1561deae.txt

[Execution Error]: Code execution failed at line 'import os' due to InterpreterError: Import of os is not allowed. Authorized imports are: ['math', 'queue', 'datetime', 're', 'itertools', 'json', 'stat', 'random', 'unicodedata', 'collections', 'statistics', 'time']


In [31]:
# Scenario 5: Output truncation (small max_output_chars)
small_output_tool = CodeExecutionTool(
    output_base_path=CODE_OUTPUT_BASE,
    embedded_tools=[],
    authorized_imports=[],
    max_output_chars=300,
)
small_output_tool.set_agent_uuid("notebook-demo-agent")
small_output = small_output_tool.get_tool()

print(small_output("""
print('A' * 2000)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_123813_91ceac83.txt


... [truncated, showing last 300 chars] ...
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
..._This content has been truncated to stay below 300 characters_...
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA



In [32]:
# Scenario 6: Missing UUID (shows warning and does not write a file)
no_uuid_tool = CodeExecutionTool(
    output_base_path=CODE_OUTPUT_BASE,
    embedded_tools=[add],
    authorized_imports=[],
    max_output_chars=1_000,
)
no_uuid_execute = no_uuid_tool.get_tool()

print(no_uuid_execute("""
print('Running without agent_uuid set')
print('add(1, 2) =', add(1, 2))
"""))


[Warning: agent_uuid is not set. The agent must inject the UUID via set_agent_uuid() before code execution can write output files.]

Running without agent_uuid set
add(1, 2) = 3



In [33]:
# Scenario 7: Agent UUID injection (no model call required)
# This demonstrates the "duck-typed" protocol: the agent sets UUID on tool instances.

try:
    from anthropic_agent import AnthropicAgent

    injected_tool = CodeExecutionTool(
        output_base_path=CODE_OUTPUT_BASE,
        embedded_tools=[add],
        authorized_imports=[],
        max_output_chars=1_000,
    )
    injected_fn = injected_tool.get_tool()

    agent = AnthropicAgent(
        system_prompt="UUID injection demo (no run)",
        tools=[injected_fn],
    )

    print("Agent UUID:", agent.agent_uuid)
    print("Tool UUID:", injected_tool.agent_uuid)
    print("UUIDs match:", agent.agent_uuid == injected_tool.agent_uuid)

except Exception as e:
    # If the notebook env doesn't have the Anthropic dependency installed,
    # importing AnthropicAgent may fail.
    print("Skipping AnthropicAgent injection demo:", type(e).__name__, str(e))


Agent UUID: 035c0214-ea79-421f-9471-bdfb136febf6
Tool UUID: 035c0214-ea79-421f-9471-bdfb136febf6
UUIDs match: True


In [34]:
# Build a CodeExecutionTool that embeds other common_tools
# Note: these tool functions were created earlier in this notebook.

code_tool_common_tools = CodeExecutionTool(
    output_base_path=CODE_OUTPUT_BASE,
    embedded_tools=[
        read_file,
        list_dir,
        glob_file_search,
        grep_search,
        apply_patch,
    ],
    authorized_imports=["json"],
    max_output_chars=2_000,
)
code_tool_common_tools.set_agent_uuid("notebook-demo-agent")
code_execute_common_tools = code_tool_common_tools.get_tool()

print("Embedded tool names:", [t.__tool_schema__["name"] for t in code_tool_common_tools.embedded_tools])

Embedded tool names: ['read_file', 'list_dir', 'glob_file_search', 'grep_search', 'apply_patch']


In [35]:
# Example A: Call list_dir from within executed code
print(code_execute_common_tools("""
result = list_dir(".")
print("Directory listing from code:")
print(result)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124836_b8617e46.txt

Directory listing from code:
temp/
   - test-agent-123/
      - api_refactor_20260115_100908.md
      - API_v2_Upgrade_20260115_100908.md
      - db_migration_20260115_100908.md
      - plan_20260115_074238.md
      - plan_20260115_074249.md
   - README.md
   - test_file.md



In [36]:
# Example B: Call glob_file_search from within executed code
print(code_execute_common_tools("""
files = glob_file_search("*.md")
print("Markdown files found:")
print(files)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124841_0726d966.txt

Markdown files found:
test-agent-123/API_v2_Upgrade_20260115_100908.md
test-agent-123/db_migration_20260115_100908.md
test-agent-123/api_refactor_20260115_100908.md
test-agent-123/plan_20260115_074249.md
test-agent-123/plan_20260115_074238.md
README.md
test_file.md



In [37]:
# Example C: Call read_file from within executed code
print(code_execute_common_tools("""
content = read_file("README.md", start_line_one_indexed=1, no_of_lines_to_read=10)
print("First 10 lines of README.md:")
print(content)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124848_dcbfe4f9.txt

First 10 lines of README.md:
[lines 1-10 of 102 in README.md]
# anthropic-agent

A small Python library for building **Anthropic-powered agents** with:
- **streaming** responses
- **tool calling** (client tools)
- **optional server tools** (passed through to Anthropic)
- **persistence** (filesystem or Postgres) and **file storage** (local or S3)

> **For sample agent configurations examples hanve a look at agent_test.ipynb**





In [38]:
# Example D: Call grep_search from within executed code
print(code_execute_common_tools("""
matches = grep_search("agent")
print("Grep results for 'agent':")
print(matches[:500])  # truncate for display
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124848_2c4a4e4d.txt

Grep results for 'agent':
README.md:
  1: # anthropic-<match>agent</match>
  2- 
  3: A small Python library for building **Anthropic-powered <match>agent</match>s** with:
  4- - **streaming** responses
  5- - **tool calling** (client tools)
  7- - **persistence** (filesystem or Postgres) and **file storage** (local or S3)
  8- 
  9: > **For sample <match>agent</match> configurations examples hanve a look at <match>agent</match>_test.ipynb**
  10- 
  11: The primary package is `anthropic_<match>agent</match>` (install na



In [39]:
# Example E: Call apply_patch from within executed code (create a file)
print(code_execute_common_tools('''
import json

patch_text = """*** Begin Patch
*** Add File: code_generated.md
+# Generated from Code
+
+This file was created by calling apply_patch from inside CodeExecutionTool.
*** End Patch"""

result = apply_patch(patch_text, dry_run=False)
print("apply_patch result:")
print(json.dumps(json.loads(result), indent=2))
'''))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124849_7ac1efd6.txt

apply_patch result:
{
  "status": "ok",
  "op": "add",
  "path": "code_generated.md",
  "hunks_applied": 0,
  "lines_added": 3,
  "lines_removed": 0,
  "dry_run": false
}



In [40]:
# Example F: Verify the file was created, then read it
print(code_execute_common_tools("""
# Verify file exists
files = glob_file_search("code_generated.md")
print("File exists:", "code_generated.md" in files)

# Read the file
content = read_file("code_generated.md")
print("File content:")
print(content)
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124850_06290c98.txt

File exists: True
File content:
[lines 1-3 of 3 in code_generated.md]
# Generated from Code

This file was created by calling apply_patch from inside CodeExecutionTool.



In [41]:
# Example G: Chained workflow - search, read, and summarize in one code block
print(code_execute_common_tools("""
# 1. Find all markdown files
md_files = glob_file_search("*.md")
file_list = [f.strip() for f in md_files.strip().split("\\n") if f.strip()]
print(f"Found {len(file_list)} markdown files")

# 2. Read first 5 lines of each (up to 3 files)
for fname in file_list[:3]:
    print(f"\\n--- {fname} ---")
    content = read_file(fname, start_line_one_indexed=1, no_of_lines_to_read=5)
    # Skip header line
    lines = content.split("\\n")[1:6]
    print("\\n".join(lines))
"""))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124852_693f170c.txt

Found 8 markdown files

--- code_generated.md ---
# Generated from Code

This file was created by calling apply_patch from inside CodeExecutionTool.

--- test-agent-123/API_v2_Upgrade_20260115_100908.md ---
# API v2 Upgrade

Upgrade notes here.


--- test-agent-123/db_migration_20260115_100908.md ---
# Database Migration Plan

## Overview
Migrate from SQLite to PostgreSQL for production.


[Last value]: ['# Database Migration Plan', '', '## Overview', 'Migrate from SQLite to PostgreSQL for production.', '']


In [42]:
# Cleanup: delete the generated file
print(code_execute_common_tools('''
import json

patch_text = """*** Begin Patch
*** Delete File: code_generated.md
*** End Patch"""

result = apply_patch(patch_text, dry_run=False)
print("Cleanup result:")
print(json.dumps(json.loads(result), indent=2))
'''))


output_path=/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp/code_tool_outputs/notebook-demo-agent/code_runs/20260119_124853_84d110d4.txt

Cleanup result:
{
  "status": "ok",
  "op": "delete",
  "path": "code_generated.md",
  "hunks_applied": 0,
  "lines_added": 0,
  "lines_removed": 3,
  "dry_run": false
}



## Tool: TodoWrite & CheckTodo

This section demonstrates the `TodoWriteTool` and `CheckTodoTool`:
- Stateless replacement semantics (every call sends the full list)
- Each todo has a unique `id` for referencing
- Persistence to YAML file scoped per agent UUID
- ID conflict detection with guidance
- Validation of required fields and status values
- Agent UUID injection via `set_agent_uuid()` / `__tool_instance__` protocol

In [3]:
import json
import shutil
from pathlib import Path

from anthropic_agent.common_tools.todo_tool import TodoWriteTool, CheckTodoTool

# Use a subdirectory of BASE_PATH for todo YAML files
TODO_BASE = BASE_PATH / "todo_data"
TODO_BASE.mkdir(parents=True, exist_ok=True)

AGENT_UUID = "todo-test-agent"

# Instantiate both tools
todo_write_tool = TodoWriteTool(base_dir=TODO_BASE)
check_todo_tool = CheckTodoTool(base_dir=TODO_BASE)

# Manually inject agent UUID (in real usage the agent does this automatically)
todo_write_tool.set_agent_uuid(AGENT_UUID)
check_todo_tool.set_agent_uuid(AGENT_UUID)

todo_write = todo_write_tool.get_tool()
check_todo = check_todo_tool.get_tool()

print("=== Schema inspection ===")
print(f"todo_write name: {todo_write.__tool_schema__['name']}")
print(f"check_todo name: {check_todo.__tool_schema__['name']}")
print(f"todo_write has __tool_instance__: {hasattr(todo_write, '__tool_instance__')}")
print(f"check_todo has __tool_instance__: {hasattr(check_todo, '__tool_instance__')}")
print(f"todo_write UUID: {todo_write_tool.agent_uuid}")
print(f"check_todo UUID: {check_todo_tool.agent_uuid}")

=== Schema inspection ===
todo_write name: todo_write
check_todo name: check_todo
todo_write has __tool_instance__: True
check_todo has __tool_instance__: True
todo_write UUID: todo-test-agent
check_todo UUID: todo-test-agent


In [4]:
# Scenario 1: Write a todo list and read it back
todos = json.dumps([
    {"id": "explore", "content": "Explore the codebase", "status": "completed", "activeForm": "Exploring the codebase"},
    {"id": "implement", "content": "Implement feature", "status": "in_progress", "activeForm": "Implementing feature"},
    {"id": "test", "content": "Write tests", "status": "pending", "activeForm": "Writing tests"},
])

print("=== Write todos ===")
print(todo_write(todos=todos))

print("\n=== Check todos ===")
print(check_todo())

=== Write todos ===
Saved 3 todo(s). (1 completed, 1 in progress, 1 pending)

=== Check todos ===
[x] explore: Explore the codebase (status: completed, activeForm: Exploring the codebase)
[~] implement: Implement feature (status: in_progress, activeForm: Implementing feature)
[ ] test: Write tests (status: pending, activeForm: Writing tests)


In [5]:
# Scenario 2: Verify YAML file on disk
yaml_path = TODO_BASE / AGENT_UUID / "todos.yaml"
print(f"YAML file exists: {yaml_path.exists()}")
print(f"YAML file path:   {yaml_path}\n")
print(yaml_path.read_text())

YAML file exists: True
YAML file path:   /Users/aurosoni/conductor/workspaces/anthropic_agent/nagoya/anthropic_agent/common_tools/temp/todo_data/todo-test-agent/todos.yaml

todos:
- id: explore
  content: Explore the codebase
  status: completed
  activeForm: Exploring the codebase
- id: implement
  content: Implement feature
  status: in_progress
  activeForm: Implementing feature
- id: test
  content: Write tests
  status: pending
  activeForm: Writing tests



In [6]:
# Scenario 3: Stateless replacement — update todo statuses
# Mark "implement" as completed, "test" as in_progress
updated_todos = json.dumps([
    {"id": "explore", "content": "Explore the codebase", "status": "completed", "activeForm": "Exploring the codebase"},
    {"id": "implement", "content": "Implement feature", "status": "completed", "activeForm": "Implementing feature"},
    {"id": "test", "content": "Write tests", "status": "in_progress", "activeForm": "Writing tests"},
])

print("=== Update todos (stateless replacement) ===")
print(todo_write(todos=updated_todos))

print("\n=== Check updated todos ===")
print(check_todo())

=== Update todos (stateless replacement) ===
Saved 3 todo(s). (2 completed, 1 in progress, 0 pending)

=== Check updated todos ===
[x] explore: Explore the codebase (status: completed, activeForm: Exploring the codebase)
[x] implement: Implement feature (status: completed, activeForm: Implementing feature)
[~] test: Write tests (status: in_progress, activeForm: Writing tests)


In [7]:
# Scenario 4: Error — duplicate IDs (with guidance)
dup_todos = json.dumps([
    {"id": "fix-auth", "content": "Fix auth bug", "status": "pending", "activeForm": "Fixing auth bug"},
    {"id": "add-tests", "content": "Add tests", "status": "pending", "activeForm": "Adding tests"},
    {"id": "fix-auth", "content": "Fix auth again", "status": "in_progress", "activeForm": "Fixing auth again"},
])

print("=== Duplicate ID error ===")
print(todo_write(todos=dup_todos))

=== Duplicate ID error ===
Error: Duplicate todo ID 'fix-auth' found at indices 0 and 2. Each todo must have a unique ID. Please rename one of them (e.g., 'fix-auth-2') and resubmit the full list.


In [8]:
# Scenario 5: Error — invalid status
bad_status = json.dumps([
    {"id": "task-1", "content": "Do something", "status": "done", "activeForm": "Doing something"},
])

print("=== Invalid status error ===")
print(todo_write(todos=bad_status))

=== Invalid status error ===
Error: Invalid status 'done' for todo 'task-1'. Must be one of: completed, in_progress, pending.


In [9]:
# Scenario 6: Error — missing required fields
missing_fields = json.dumps([
    {"id": "task-1", "content": "Do something"},
])

print("=== Missing fields error ===")
print(todo_write(todos=missing_fields))

=== Missing fields error ===
Error: Todo at index 0 is missing required fields: activeForm, status. Each todo must have: id (string), content (string), status (string), activeForm (string).


In [11]:
# Scenario 7: Error — invalid JSON
print("=== Invalid JSON error ===")
print(todo_write(todos="not valid json [}"))

=== Invalid JSON error ===
Error: Failed to parse todos JSON: Expecting value: line 1 column 1 (char 0). Please check JSON syntax — todos must be a JSON array of objects.


In [12]:
# Scenario 8: Error — missing agent UUID
no_uuid_write = TodoWriteTool(base_dir=TODO_BASE)
no_uuid_check = CheckTodoTool(base_dir=TODO_BASE)
# Deliberately NOT calling set_agent_uuid()

no_uuid_write_fn = no_uuid_write.get_tool()
no_uuid_check_fn = no_uuid_check.get_tool()

print("=== Write without UUID ===")
print(no_uuid_write_fn(todos='[{"id":"x","content":"X","status":"pending","activeForm":"X"}]'))

print("\n=== Check without UUID ===")
print(no_uuid_check_fn())

=== Write without UUID ===
Error: agent_uuid is not set. Cannot save todos without an agent session.

=== Check without UUID ===
Error: agent_uuid is not set. Cannot read todos without an agent session.


In [13]:
# Scenario 9: Check empty state (nonexistent agent)
empty_check = CheckTodoTool(base_dir=TODO_BASE)
empty_check.set_agent_uuid("nonexistent-agent-uuid")
empty_check_fn = empty_check.get_tool()

print("=== Check todos for nonexistent agent ===")
print(empty_check_fn())

=== Check todos for nonexistent agent ===
No todos found for this agent.


In [14]:
# Scenario 10: Agent UUID injection via AnthropicAgent (duck-typed protocol)
try:
    from anthropic_agent import AnthropicAgent

    injected_write = TodoWriteTool(base_dir=TODO_BASE)
    injected_check = CheckTodoTool(base_dir=TODO_BASE)
    write_fn = injected_write.get_tool()
    check_fn = injected_check.get_tool()

    agent = AnthropicAgent(
        system_prompt="Todo tool UUID injection demo (no run)",
        tools=[write_fn, check_fn],
    )

    print("Agent UUID:     ", agent.agent_uuid)
    print("Write tool UUID:", injected_write.agent_uuid)
    print("Check tool UUID:", injected_check.agent_uuid)
    print("UUIDs match:    ", agent.agent_uuid == injected_write.agent_uuid == injected_check.agent_uuid)

except Exception as e:
    print("Skipping AnthropicAgent injection demo:", type(e).__name__, str(e))

Agent UUID:      bbc065a1-5b82-4ded-bce7-79a51aee7300
Write tool UUID: bbc065a1-5b82-4ded-bce7-79a51aee7300
Check tool UUID: bbc065a1-5b82-4ded-bce7-79a51aee7300
UUIDs match:     True


In [15]:
# Cleanup: remove todo test data
shutil.rmtree(TODO_BASE / AGENT_UUID, ignore_errors=True)
print("Cleaned up todo test data.")

Cleaned up todo test data.


## Tool: Plan Mode (EnterPlanMode, ExitPlanMode, CreatePlan, EditPlan)

This section demonstrates the four plan-mode tools:
- **EnterPlanModeTool / ExitPlanModeTool**: Frontend-only tools (schema + docstring, no backend logic)
- **CreatePlanTool**: Creates a plan with title, markdown overview, and YAML todos; returns a `plan_id`
- **EditPlanTool**: Edits an existing plan's overview and/or todos using apply_patch hunk format

Key behaviors tested:
- Frontend tools have `__tool_executor__ == "frontend"`
- Plan persistence as YAML files scoped per agent UUID
- Plan ID generation (slugified title + short UUID suffix)
- Todo validation (same rules as TodoWriteTool)
- EditPlan patching via apply_patch hunks (without file headers)
- Error handling: missing UUID, nonexistent plan, invalid YAML, validation failures

In [20]:
import json
import shutil
from pathlib import Path

from anthropic_agent.common_tools.plan_tools import (
    EnterPlanModeTool,
    ExitPlanModeTool,
    CreatePlanTool,
    EditPlanTool,
)

# Use a subdirectory of BASE_PATH for plan YAML files
PLAN_BASE = BASE_PATH / "plan_data"
PLAN_BASE.mkdir(parents=True, exist_ok=True)

PLAN_AGENT_UUID = "plan-test-agent"

# Instantiate backend tools
create_plan_tool = CreatePlanTool(base_dir=PLAN_BASE)
edit_plan_tool = EditPlanTool(base_dir=PLAN_BASE)

# Manually inject agent UUID (in real usage the agent does this automatically)
create_plan_tool.set_agent_uuid(PLAN_AGENT_UUID)
edit_plan_tool.set_agent_uuid(PLAN_AGENT_UUID)

create_plan = create_plan_tool.get_tool()
edit_plan = edit_plan_tool.get_tool()

# Instantiate frontend tools (no UUID needed — they have no backend logic)
enter_plan_mode_tool = EnterPlanModeTool()
exit_plan_mode_tool = ExitPlanModeTool()

enter_plan_mode = enter_plan_mode_tool.get_tool()
exit_plan_mode = exit_plan_mode_tool.get_tool()

print("=== Schema inspection ===")
print(f"create_plan name: {create_plan.__tool_schema__['name']}")
print(f"edit_plan name:   {edit_plan.__tool_schema__['name']}")
print(f"enter_plan_mode name: {enter_plan_mode.__tool_schema__['name']}")
print(f"exit_plan_mode name:  {exit_plan_mode.__tool_schema__['name']}")
print(f"\ncreate_plan has __tool_instance__: {hasattr(create_plan, '__tool_instance__')}")
print(f"edit_plan has __tool_instance__:   {hasattr(edit_plan, '__tool_instance__')}")
print(f"create_plan UUID: {create_plan_tool.agent_uuid}")
print(f"edit_plan UUID:   {edit_plan_tool.agent_uuid}")

=== Schema inspection ===
create_plan name: create_plan
edit_plan name:   edit_plan
enter_plan_mode name: enter_plan_mode
exit_plan_mode name:  exit_plan_mode

create_plan has __tool_instance__: True
edit_plan has __tool_instance__:   True
create_plan UUID: plan-test-agent
edit_plan UUID:   plan-test-agent


In [21]:
# Scenario 1: Frontend tools — verify executor is "frontend" and schemas look correct
print("=== EnterPlanMode ===")
print(f"  executor: {enter_plan_mode.__tool_executor__}")
print(f"  params:   {enter_plan_mode.__tool_schema__['input_schema']['properties']}")
print(f"  required: {enter_plan_mode.__tool_schema__['input_schema'].get('required', [])}")

print("\n=== ExitPlanMode ===")
print(f"  executor: {exit_plan_mode.__tool_executor__}")
print(f"  params:   {list(exit_plan_mode.__tool_schema__['input_schema']['properties'].keys())}")
print(f"  required: {exit_plan_mode.__tool_schema__['input_schema'].get('required', [])}")

=== EnterPlanMode ===
  executor: frontend
  params:   {}
  required: []

=== ExitPlanMode ===
  executor: frontend
  params:   ['plan_id']
  required: ['plan_id']


In [22]:
# Scenario 2: CreatePlan — create a plan and inspect the returned plan_id
todos_yaml = """- id: research
  content: Research OAuth2 libraries
  status: pending
  activeForm: Researching OAuth2 libraries
- id: implement
  content: Implement OAuth2 flow
  status: pending
  activeForm: Implementing OAuth2 flow
- id: test
  content: Write integration tests
  status: pending
  activeForm: Writing integration tests"""

plan_id = create_plan(
    title="Auth System Refactor",
    overview="## Goal\n\nRefactor the auth system to support OAuth2.\n\n## Approach\n\n1. Research libraries\n2. Implement flow\n3. Write tests",
    todos=todos_yaml,
)

print("=== CreatePlan result ===")
print(f"plan_id: {plan_id}")
print(f"plan_id starts with 'auth-system-refactor-': {plan_id.startswith('auth-system-refactor-')}")
print(f"plan_id suffix length (6-char hex): {len(plan_id.split('-')[-1])}")

=== CreatePlan result ===
plan_id: auth-system-refactor-872700
plan_id starts with 'auth-system-refactor-': True
plan_id suffix length (6-char hex): 6


In [23]:
# Scenario 3: Verify YAML file on disk
plan_path = PLAN_BASE / PLAN_AGENT_UUID / "plans" / f"{plan_id}.yaml"
print(f"YAML file exists: {plan_path.exists()}")
print(f"YAML file path:   {plan_path}\n")
print(plan_path.read_text())

YAML file exists: True
YAML file path:   /Users/aurosoni/conductor/workspaces/anthropic_agent/nagoya/anthropic_agent/common_tools/temp/plan_data/plan-test-agent/plans/auth-system-refactor-872700.yaml

plan_id: auth-system-refactor-872700
title: Auth System Refactor
overview: '## Goal


  Refactor the auth system to support OAuth2.


  ## Approach


  1. Research libraries

  2. Implement flow

  3. Write tests'
todos:
- id: research
  content: Research OAuth2 libraries
  status: pending
  activeForm: Researching OAuth2 libraries
- id: implement
  content: Implement OAuth2 flow
  status: pending
  activeForm: Implementing OAuth2 flow
- id: test
  content: Write integration tests
  status: pending
  activeForm: Writing integration tests



In [24]:
# Scenario 4: EditPlan — patch the todos section (mark research as completed)
todos_patch = """@@
 - id: research
   content: Research OAuth2 libraries
-  status: pending
+  status: completed
   activeForm: Researching OAuth2 libraries"""

result = edit_plan(plan_id=plan_id, todos_patch=todos_patch)
print("=== EditPlan (todos patch) ===")
print(result)

# Verify the change
import yaml
with open(plan_path) as f:
    data = yaml.safe_load(f)
print(f"\nresearch todo status: {data['todos'][0]['status']}")
print(f"implement todo status: {data['todos'][1]['status']}")

=== EditPlan (todos patch) ===
Plan 'auth-system-refactor-872700' updated successfully. Patched: todos.

research todo status: completed
implement todo status: pending


In [8]:
# Scenario 5: EditPlan — patch the overview section
overview_patch = """@@
-overview: "## Goal\\n\\nRefactor the auth system to support OAuth2.\\n\\n## Approach\\n\\n\
+overview: "## Goal\\n\\nRefactor the auth system to support OAuth2 and SAML.\\n\\n## Approach\\n\\n\
"""

result = edit_plan(plan_id=plan_id, overview_patch=overview_patch)
print("=== EditPlan (overview patch) ===")
print(result)

# Verify
with open(plan_path) as f:
    data = yaml.safe_load(f)
print(f"\nUpdated overview:\n{data['overview']}")

=== EditPlan (overview patch) ===
Error applying overview patch: Hunk 1 failed: could not find matching context
Hint: Context mismatch. Re-read the file to get current content.
Expected context:
  overview: "## Goal\n\nRefactor the auth system to support OAuth2.\n\n## Approach\n\n+overview: "## Goal\n\nRefactor the auth system to support OAuth2 and SAML.\n\n## Approach\n\n

File content near line 1:
  1: plan_id: auth-system-refactor-a9b2e6
  2: title: Auth System Refactor
  3: overview: '## Goal
  4: 
  5: 
  6:   Refactor the auth system to support OAuth2.
  7: 
  8: 

Updated overview:
## Goal

Refactor the auth system to support OAuth2.

## Approach

1. Research libraries
2. Implement flow
3. Write tests


In [9]:
# Scenario 6: Multiple plans per agent — create a second plan
plan_id_2 = create_plan(
    title="Database Migration",
    overview="Migrate from SQLite to PostgreSQL.",
    todos="""- id: schema
  content: Design new schema
  status: pending
  activeForm: Designing new schema""",
)

print("=== Second plan created ===")
print(f"plan_id_2: {plan_id_2}")

# List all plan files for this agent
plans_dir = PLAN_BASE / PLAN_AGENT_UUID / "plans"
plan_files = sorted(plans_dir.glob("*.yaml"))
print(f"\nAll plans for agent ({len(plan_files)}):")
for pf in plan_files:
    print(f"  - {pf.name}")

=== Second plan created ===
plan_id_2: database-migration-37dd04

All plans for agent (2):
  - auth-system-refactor-a9b2e6.yaml
  - database-migration-37dd04.yaml


In [10]:
# Scenario 7: Error — missing agent UUID
no_uuid_create = CreatePlanTool(base_dir=PLAN_BASE)
no_uuid_edit = EditPlanTool(base_dir=PLAN_BASE)
# Deliberately NOT calling set_agent_uuid()

no_uuid_create_fn = no_uuid_create.get_tool()
no_uuid_edit_fn = no_uuid_edit.get_tool()

print("=== CreatePlan without UUID ===")
print(no_uuid_create_fn(title="Test", overview="Test", todos="- id: x\n  content: X\n  status: pending\n  activeForm: X"))

print("\n=== EditPlan without UUID ===")
print(no_uuid_edit_fn(plan_id="fake-id", todos_patch="@@\n-old\n+new"))

=== CreatePlan without UUID ===
Error: agent_uuid is not set. Cannot create plan without an agent session.

=== EditPlan without UUID ===
Error: agent_uuid is not set. Cannot edit plan without an agent session.


In [11]:
# Scenario 8: Error — invalid YAML for todos
print("=== Invalid YAML error ===")
print(create_plan(title="Bad Plan", overview="Test", todos="not: valid: yaml: [}"))

=== Invalid YAML error ===
Error: Failed to parse todos YAML: mapping values are not allowed here
  in "<unicode string>", line 1, column 11:
    not: valid: yaml: [}
              ^. Please check YAML syntax — todos must be a YAML list of objects with id, content, status, and activeForm fields.


In [12]:
# Scenario 9: Error — todos not a list (single object instead of list)
print("=== Todos not a list error ===")
print(create_plan(
    title="Bad Plan",
    overview="Test",
    todos="id: task-1\ncontent: Do something\nstatus: pending\nactiveForm: Doing something",
))

=== Todos not a list error ===
Error: Todos must be a YAML list of objects. Got a single value instead.


In [13]:
# Scenario 10: Error — missing required fields in todos
print("=== Missing todo fields error ===")
print(create_plan(
    title="Bad Plan",
    overview="Test",
    todos="- id: task-1\n  content: Do something",
))

=== Missing todo fields error ===
Error: Todo at index 0 is missing required fields: activeForm, status. Each todo must have: id (string), content (string), status (string), activeForm (string).


In [14]:
# Scenario 11: Error — invalid status in todos
print("=== Invalid status error ===")
print(create_plan(
    title="Bad Plan",
    overview="Test",
    todos="- id: task-1\n  content: Do something\n  status: done\n  activeForm: Doing something",
))

=== Invalid status error ===
Error: Invalid status 'done' for todo 'task-1'. Must be one of: completed, in_progress, pending.


In [15]:
# Scenario 12: Error — empty title
print("=== Empty title error ===")
print(create_plan(title="  ", overview="Test", todos="- id: x\n  content: X\n  status: pending\n  activeForm: X"))

=== Empty title error ===
Error: Plan title cannot be empty.


In [16]:
# Scenario 13: Error — EditPlan on nonexistent plan
print("=== Nonexistent plan error ===")
print(edit_plan(plan_id="nonexistent-plan-abc123", todos_patch="@@\n-old\n+new"))

=== Nonexistent plan error ===
Error: Plan 'nonexistent-plan-abc123' not found. Expected file at: /Users/aurosoni/conductor/workspaces/anthropic_agent/nagoya/anthropic_agent/common_tools/temp/plan_data/plan-test-agent/plans/nonexistent-plan-abc123.yaml


In [17]:
# Scenario 14: Error — EditPlan with no patches provided
print("=== No patches error ===")
print(edit_plan(plan_id=plan_id))

=== No patches error ===
Error: No patches provided. Supply at least one of overview_patch or todos_patch.


In [18]:
# Scenario 15: Agent UUID injection via AnthropicAgent (duck-typed protocol)
try:
    from anthropic_agent import AnthropicAgent

    injected_create = CreatePlanTool(base_dir=PLAN_BASE)
    injected_edit = EditPlanTool(base_dir=PLAN_BASE)
    create_fn = injected_create.get_tool()
    edit_fn = injected_edit.get_tool()

    agent = AnthropicAgent(
        system_prompt="Plan tool UUID injection demo (no run)",
        tools=[create_fn, edit_fn],
    )

    print("Agent UUID:       ", agent.agent_uuid)
    print("CreatePlan UUID:  ", injected_create.agent_uuid)
    print("EditPlan UUID:    ", injected_edit.agent_uuid)
    print("UUIDs match:      ", agent.agent_uuid == injected_create.agent_uuid == injected_edit.agent_uuid)

except Exception as e:
    print("Skipping AnthropicAgent injection demo:", type(e).__name__, str(e))

Agent UUID:        34639beb-fe5e-490a-8ac2-46e2d58df0eb
CreatePlan UUID:   34639beb-fe5e-490a-8ac2-46e2d58df0eb
EditPlan UUID:     34639beb-fe5e-490a-8ac2-46e2d58df0eb
UUIDs match:       True


In [19]:
# Cleanup: remove plan test data
shutil.rmtree(PLAN_BASE / PLAN_AGENT_UUID, ignore_errors=True)
print("Cleaned up plan test data.")

Cleaned up plan test data.
